Beispiel mit nsenter

In [ ]:
! kubectl run webshop --restart=Never --image=registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0

In [ ]:
! pstree -npTA

In [ ]:
! ip addr

In [ ]:
! cat /etc/issue

Jetzt die gleichen Befehle innerhalb unseres Containers `webshop`

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --net ip addr

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) -a pstree -p

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) -a cat /etc/issue

In [ ]:
! kubectl exec pod/webshop -- ip addr
! echo "----------------------------------"
! kubectl exec pod/webshop -- pstree -p
! echo "----------------------------------"
! kubectl exec pod/webshop -- cat /etc/issue

Anzeige der Images

In [ ]:
! kubectl delete pod/webshop

In [ ]:
! pgrep -o -x webshop

In [ ]:
! microk8s ctr image list | awk '{print $1, $4, $5, $6}' | grep shop